In [71]:
import librosa
import neuro
import random
import numpy as np
import pandas as pd
import risp
import eons
import json
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import wave
import os 
from scipy.signal import find_peaks

# original params
if False:
    eo_params = {
        "starting_nodes": 3,
        "starting_edges": 6,
        "merge_rate": 0,
        "population_size": 100,
        "multi_edges": 0,
        "crossover_rate": 0.5,
        "mutation_rate": 0.9,
        "selection_type": "tournament",
        "tournament_size_factor": 0.1,
        "tournament_best_net_factor": 0.9,
        "random_factor": 0.05,
        "num_mutations": 3,
        "node_mutations": { "Threshold": 1.0 },
        "net_mutations": { },
        "edge_mutations": { "Weight": 0.5, "Delay": 0.5 },
        "num_best" : 4
    }

eo_params = {
    "starting_nodes": 3,
    "starting_edges": 6,
    "merge_rate": 0.1,
    "population_size": 100,
    "multi_edges": 0,
    "crossover_rate": 0.5,
    "mutation_rate": .9,
    "selection_type": "tournament",
    "tournament_size_factor": 0.1,
    "tournament_best_net_factor": 0.9,
    "random_factor": 0.05,
    "num_mutations": 20,
    "node_mutations": { "Threshold": 1.0 },
    "net_mutations": { },
    "edge_mutations": { "Weight": 0.5, "Delay": 0.5 },
    "num_best" : 4
}



In [76]:
unique_sample_rates = np.array([]) # <-------------



import wave
import librosa
import numpy as np
import os 
from scipy.signal import find_peaks

directory = 'audio_database'
def find_peaks_per_channel(spectrum, threshold = 0):
    peaks = np.zeros_like(spectrum)
    
    for i in range(spectrum.shape[0]):
        channel_data = spectrum[i, :]
        channel_peaks, _ = find_peaks(channel_data, height=threshold)
        peaks[i, channel_peaks] = 1
    
    return peaks

# Create a new folder to save the npy files
new_folder = "npy_files"
new_folder_path = os.path.join(directory, new_folder)
os.makedirs(new_folder_path, exist_ok=True)

for folder_name in os.listdir(directory):

    unique_song_len = np.array([]) #<---------------
    

    folder_path = os.path.join(directory, folder_name)
    if os.path.isdir(folder_path):  # Check if the item is a directory
        for filename in os.listdir(folder_path):
            if filename.endswith('.wav'):
                try: 
                    file_path = os.path.join(folder_path, filename)
                    audio_signal, sample_rate = librosa.load(file_path, sr=None) # we may want to hardcode our sampling rate if it is not consistant between songs 
                    
                    unique_sample_rates = np.append(unique_sample_rates,sample_rate) # <----------
                    unique_song_len = np.append(unique_song_len,np.shape(audio_signal)) # <----------

                    n_fft = 1024
                    hop_length = n_fft // 16 #number of audio samples between successive frames in the Short-Time Fourier Transform (STFT) analysis
                    magnitude_spectrum = np.abs(librosa.stft(audio_signal, n_fft=n_fft, hop_length=hop_length))

                    # Appling Mel filterbank from librosa 
                    num_mels = 20 #128 originally, but likely too much computation time and no longer actually follows any biological model
                    mel_spectrum = librosa.feature.melspectrogram(
                        sr=sample_rate,
                        S=magnitude_spectrum,
                        n_fft=n_fft,
                        hop_length=hop_length,
                        n_mels=num_mels
                    )
                    
                    peak_spectrogram = find_peaks_per_channel(mel_spectrum)

                    
                    
                    # Saving peak_spectrogram with a filename including the original file's name
                    # this will overwrite any pre-existing files with the same name
                    output_filename = f"{filename}_peak_spectrogram.npy"
                    output_path = os.path.join(new_folder_path, output_filename)
                    np.save(output_path, peak_spectrogram)
                    
                    #print(f"processed {filename} in folder {folder_name}. Saved peak_spectrogram as {output_filename}")

                    
                
                except Exception as e:
                    print(f"Error processing {filename} in folder {folder_name}: {str(e)}")
                    continue
        print(f"unique song length in {folder_name} is {np.unique(unique_song_len)}") # <------------
print(f"unique sample rates for all genres {np.unique(unique_sample_rates)}")

unique song length in pop is [661504.]
unique song length in metal is [661504. 661794.]
unique song length in disco is [661344. 661504. 661676. 661760. 661794. 663520. 664180. 665060. 666160.
 667920. 668140.]
unique song length in npy_files is []
unique song length in blues is [661794.]
unique song length in reggae is [661504. 661794.]
unique song length in classical is [661344. 661408. 661676. 661760. 661794. 663080. 663520. 665280. 669680.
 670120. 672282.]
unique song length in rock is [661408. 661500. 661794. 667920. 669460. 670340.]
unique song length in hiphop is [660000. 661408. 661504. 661676. 661760. 661794. 664400. 665280. 667700.
 668140. 669240. 669680. 675808.]
unique song length in country is [661100. 661408. 661760. 661794. 663300. 663740. 666820. 668800. 669680.]
Error processing jazz.00054.wav in folder jazz: 
unique song length in jazz is [661676. 661794. 661980. 665280. 665940. 666820. 667480. 669240. 672100.]
unique sample rates for all genres [22050.]


In [78]:
def load_npy_files_with_prefix(directory, prefix):
    npy_files = [file for file in os.listdir(directory) if file.startswith(prefix) and file.endswith('.npy')]
    npy_files.sort()  # Sort the files for consistent order
    
    if len(npy_files) == 0:
        raise ValueError(f"No npy files found with prefix '{prefix}' in directory '{directory}'")
    loaded_data = []
    
    for npy_file in npy_files:
        npy_path = os.path.join(directory, npy_file)
        data = np.load(npy_path)
        loaded_data.append(data)
    
    return np.array(loaded_data)

In [74]:

'''
directory = new_folder_path
prefix = 'blues'

try:
    data_array = load_npy_files_with_prefix(directory, prefix)
    print(f"Loaded {len(data_array)} npy files with prefix '{prefix}'")
    print("Shape of the loaded array:", data_array.shape)
except ValueError as e:
    print(str(e))
data_array
'''


'\ndirectory = new_folder_path\nprefix = \'blues\'\n\ntry:\n    data_array = load_npy_files_with_prefix(directory, prefix)\n    print(f"Loaded {len(data_array)} npy files with prefix \'{prefix}\'")\n    print("Shape of the loaded array:", data_array.shape)\nexcept ValueError as e:\n    print(str(e))\ndata_array\n'

In [80]:
directory = new_folder_path


# loading hiphop songs 
X = load_npy_files_with_prefix(directory, 'pop')





In [ ]:
# Defining training and testing data 
X = np.array(spike_df.get('major_data'))
X = np.append(X, np.array(spike_df.get('minor_data')))
y = np.tile([1,0],(13,1))
y = np.append(y, np.tile([0,1],(13,1)),axis=0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=7)

#labels = sorted(np.unique(y_train))
labels = [[1,0],[0,1]]

dmin = [np.min(X_train[i]) for i in range(X_train.shape[0])]
dmax = [np.max(X_train[i]) for i in range(X_train.shape[0])]



NameError: name 'spike_df' is not defined

'audio_database/npy_files'

# selecting sample scale 
for i in range(len(X_train)):
    # mel region (20 total) corresponds to id 
    for j in range(len(X_train[i])):
        # time bin selection 
        for k in range(len(X_train[i][j])): 
            if X_train[i][j][k] != 0:
                spike = neuro.Spike(id=j,time=0,value=X_train[i][j][k])
                proc .... 


In [ ]:
risp_config = {
  "leak_mode": True,
  "min_weight": -1,
  "max_weight": 1,
  "min_threshold": -1,
  "max_threshold": 1,
  "max_delay": 5
}

proc = risp.Processor(risp_config)

temp_net = neuro.Network()
temp_net.set_properties(proc.get_network_properties())

In [ ]:
def create_neuron(neuron_id, net, moa):
    neuron = net.add_node(neuron_id)
    temp_net.randomize_node_properties(moa, neuron)
    return neuron

In [ ]:
n_inputs = 20
n_hidden = 40
n_outputs = len(labels)
n_neurons = n_inputs+n_hidden+n_outputs
n_synapses = 100
seed = 42

moa = neuro.MOA()
moa.seed(seed)
random.seed(seed)

In [ ]:
if False:
    for i in range(n_inputs):
        neuron = create_neuron(i, temp_net, moa)
        neuron.set("Threshold",0.75)
        temp_net.add_input(i)

    for i in range(n_outputs):
        neuron = create_neuron(i+n_inputs, temp_net, moa)
        neuron.set("Threshold",0.75)
        temp_net.add_output(i)
        
    for i in range(n_hidden):
        neuron = create_neuron(i+n_inputs+n_outputs, temp_net, moa)
        
#if False:
for i in range(n_inputs):
    neuron = create_neuron(i, temp_net, moa)
    neuron.set("Threshold",0.75)
    temp_net.add_input(neuron.id)
    
for i in range(n_outputs):
    neuron = create_neuron(i+n_inputs, temp_net, moa)
    neuron.set("Threshold",0.75)
    temp_net.add_output(neuron.id)
    
for i in range(n_hidden):
    neuron = create_neuron(i+n_inputs+n_outputs, temp_net, moa)




In [ ]:
for i in range(n_synapses):
    source = random.randint(0,n_neurons-1)
    dest = random.randint(0,n_neurons-1)
    synapse = temp_net.add_or_get_edge(source, dest)
    temp_net.randomize_edge_properties(moa, synapse)

In [ ]:
evolver = eons.EONS(eo_params)
evolver.set_template_network(temp_net)

pop = evolver.generate_population(eo_params,1)

# included into fitness function 
def get_prediction(X):
    proc.clear_activity()
        # selecting sample scale 
    for i in range(len(X_train)):
        # mel region (20 total) corresponds to id 
        for j in range(len(X_train[i])):
            # time bin selection 
            for k in range(len(X_train[i][j])): 
                if X_train[i][j][k] != 0:
                    spike = neuro.Spike(id=j,time=0,value=X_train[i][j][k])
                    proc.apply_spike(spike)
    proc.run(100)
    return labels[proc.output_count_max(n_outputs)[0]]

def fitness(net, X, y):
    proc.load_network(net)
    proc.clear_activity()
    for l in range(net.num_nodes()):
        proc.track_neuron_events(l)
    
        # selecting sample scale 
    for i in range(len(X_train)):
        # mel region (20 total) corresponds to id 
        for j in range(len(X_train[i])):
            # time bin selection 
            for k in range(len(X_train[i][j])): 
                if X_train[i][j][k] != 0:
                    spike = neuro.Spike(id=j,time=0,value=X_train[i][j][k])
                    proc.apply_spike(spike)
        #proc.run(100)
        y_predict = labels[proc.output_count_max(n_outputs)[0]]
        
    
        #y_predict = [get_prediction(x) for x in X]
        return accuracy_score(y_predict, y[i])

In [ ]:
def get_prediction(x):
    #print(x)
    #proc.clear_activity()
    for j in range(len(x)):
            # time bin selection 
            for k in range(len(x[j])): 
                if x[j][k] != 0:
                    spike = neuro.Spike(id=j,time=0,value=x[j][k])
                    proc.apply_spike(spike)
    proc.run(50)
    return labels[proc.output_count_max(n_outputs)[0]]

In [ ]:
def fitness(net, X, y):
    proc.load_network(net)
    
    # Set up output tracking
    for i in range(n_outputs):
        proc.track_neuron_events(i)
    
    y_predict = [get_prediction(x) for x in X]
    #print(len(y_predict))
    return accuracy_score(y_predict, y)

In [ ]:
vals = []
for i in range(100):
    # Calculate the fitnesses of all of the networks in the population
    fitnesses = [fitness(net.network, X_train, y_train) for net in pop.networks]
    
    # Track the best performing network throughout and print the current best result
    max_fit = max(fitnesses)
    mean_fit = np.mean(fitnesses)
    #print(fitnesses)
    vals.append(max_fit)
    print("Epoch ", i, " : ","max fit", max_fit, "mean fit",mean_fit)
    
    # Create the next population based on the fitnesses of the current population
    pop = evolver.do_epoch(pop, fitnesses, eo_params)

Epoch  0  :  max fit 0.5882352941176471 mean fit 0.4205882352941177
Epoch  1  :  max fit 0.5882352941176471 mean fit 0.45000000000000007
Epoch  2  :  max fit 0.5882352941176471 mean fit 0.49764705882352944
Epoch  3  :  max fit 0.5882352941176471 mean fit 0.5229411764705881
Epoch  4  :  max fit 0.5882352941176471 mean fit 0.5458823529411764
Epoch  5  :  max fit 0.5882352941176471 mean fit 0.5458823529411764
Epoch  6  :  max fit 0.5882352941176471 mean fit 0.5494117647058823
Epoch  7  :  max fit 0.5882352941176471 mean fit 0.5423529411764706
Epoch  8  :  max fit 0.5882352941176471 mean fit 0.5405882352941176
Epoch  9  :  max fit 0.5882352941176471 mean fit 0.5294117647058821
Epoch  10  :  max fit 0.5882352941176471 mean fit 0.5617647058823527
Epoch  11  :  max fit 0.5882352941176471 mean fit 0.5517647058823529
Epoch  12  :  max fit 0.5882352941176471 mean fit 0.5529411764705882
Epoch  13  :  max fit 0.5882352941176471 mean fit 0.5617647058823528
Epoch  14  :  max fit 0.9411764705882353 m

In [ ]:

best_net = pop.networks[fitnesses.index(max_fit)].network
train = fitness(best_net, X_train, y_train)
print("Training Accuracy: ", train)
test = fitness(best_net, X_test, y_test)
print("Testing Accuracy: ", test)

Training Accuracy:  0.9411764705882353
Testing Accuracy:  0.3333333333333333
